In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import requests
import pandas as pd
import geopandas as gpd

In [ ]:
"""
You need an authorization key, I got a trial one by mailing: DBOpenData@deutschebahn.com
For real stuff I think you have to register: 

Für einen eigenen API-Key bitte im Developer-Portal registrieren: https://developer.deutschebahn.com/store/getting-started 
Und die Fahrplan-API abonnieren: https://developer.deutschebahn.com/store/apis/info?name=Fahrplan&version=v1&provider=DBOpenData

"""

#My local key:
with open('DB_API_key.txt','r') as file:
    API_Key =  file.readline()

In [ ]:
params = {}
params['lang'] = 'en'
params['authKey'] = API_Key
params['input'] = 'Düsseldorf Hbf'
params['format'] ='json'
r = requests.get('https://open-api.bahn.de/bin/rest.exe/location.name',params = params)
result = r.json()
station = result['LocationList']['StopLocation'][0]
station

In [ ]:

params = {}
params['lang'] = 'en'
params['authKey'] = 'DBhackFrankfurt0316'
params['id'] = station['id']
params['date'] = '2017-10-14'
params['time'] = '20%0a30'
params['format'] ='json'
r = requests.get('https://open-api.bahn.de/bin/rest.exe/departureBoard',params = params)
result = r.json()
departure_board = result['DepartureBoard']['Departure']

In [ ]:
df_departure_board = pd.DataFrame(departure_board)
df_departure_board.get_value(0,'JourneyDetailRef')

In [ ]:
df_departure_board = pd.DataFrame(departure_board)
df_departure_board

In [ ]:
"""Correcting some errors in the API output"""
def merge_cols(df,target_col,other_col):
    if other_col in df:
        df[target_col] = df[target_col].fillna(df[other_col])
        df.drop(other_col,axis = 1,inplace = True)    
 
merge_cols(df_departure_board,'type','tyte')
merge_cols(df_departure_board,'type','tyoe')


In [ ]:
df_departure_board.to_csv('departure_board.csv',sep = ';')

In [ ]:
df_departure_board.ix[:,df_departure_board.columns != 'JourneyDetailRef']

In [ ]:
index = df_departure_board['name'] == 'ICE 26'
line = df_departure_board.loc[index].iloc[0]


#This is how you get data about a specific journey:
r = requests.get(line['JourneyDetailRef']['ref'])
journey = r.json()['JourneyDetail']
df_journey = pd.DataFrame(journey['Stops']['Stop'])
df_journey

In [ ]:
df_journey['lat'] = pd.to_numeric(df_journey['lat'])
df_journey['lon'] = pd.to_numeric(df_journey['lon'])

In [ ]:
from shapely.geometry import Point

geometry = [Point(xy) for xy in zip(df_journey.lon, df_journey.lat)]
df = df_journey.drop(['lon', 'lat'], axis=1)
crs = {'init': 'epsg:4326'}
geo_df = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)

In [ ]:
fig,ax = plt.subplots()
geo_df.plot(ax = ax)

In [ ]:
df_world_map = gpd.read_file(filename = 'simplified-land-polygons-complete-3857/simplified_land_polygons.shp')

In [ ]:
help(df_world_map.query)